# Тестирование гипотез в Python

In [ ]:
import pandas as pd
import scipy.stats as scs
import statsmodels.stats.diagnostic as sm
import seaborn as sns
import pingouin as pg
df = pd.read_csv('sleep75.csv')
df.shape # Размер датасета

Основные переменные в датафрейме `df`
- `sleep`: продолжительность сна (мин/нед)
- `totwrk`: рабочее время (мин/нед)
- `age`: возраст (в годах)
- `educ`: уровень образрвания в годах
- `male`: гендерный фактор (бинарная, =1 для мужчин)
- `south`: географический фактор (бинарная, =1 если живёт на юге)

In [ ]:
df.shape[0] # Число наблюдений

## Одновыборочный t-тест
Для переменной `sleep` будем тестировать гипотезу $H_0:\mu=3200$
Уровень значимости выберем 5%

In [ ]:
# Первый способ с использованием библиотеки scipy.stats
t_stat, p_val=scs.ttest_1samp(a=df['sleep'], popmean=3200)
t_stat, p_val

5%-критическое значение $t$-распределения со степенями свободы $n-1=705$

In [ ]:
scs.t.ppf( q=1-0.05/2, df=df['sleep'].shape[0]-1 )

In [ ]:
# Второй способ с использованием бибиотеки pengouin
pg.ttest(x=df['sleep'], y=3200)

**Вывод**: на выбранно уровне значимости тестируемая гипотеза отвергается ($|t|>t_{cr}$, p_val<5\%)

## Двухвыборочный t-тест
Разделим наблюдения по переменной `sleep` на две подвыборки относительно бинарной переменной `male`.
Будем тестировать гипотезу о равенстве средних в генеральной совокупности $H_0:\mu_{X}=\mu_{Y}$ при **допущении неравенства дисперсий**. Уровень значимости выберем 1%

In [ ]:
# Первый способ с использованием библиотеки scipy.stats
X = df[ df['male']==1 ]['sleep']
Y = df[ df['male']==0 ]['sleep']
t_stat, p_val=scs.ttest_ind(a=X, b=Y, equal_var=False)
t_stat, p_val

In [ ]:
# Второй способ с использованием бибиотеки pengouin
pg.ttest(x=X,y=Y)

1%-критическое значение $t$-распределения со степенями свободы $$df=\frac{\left(s_X^2/n+s_Y^2/m\right)^2}{\frac{(s^2_X/n)^2}{n-1}+\frac{(s^2_Y/m)^2}{m-1}} $$

In [ ]:
s_X=X.std()
s_Y=Y.std()
n=X.shape[0]
m=Y.shape[0]
# степени свободы
dof=((s_X**2)/n+(s_Y**2)/m)**2 /( ((s_X**2)/n)**2/(n-1)+((s_Y**2)/m)**2/(m-1) )
# критическое значение t-распределения
t_cr=scs.t.ppf( q=1-0.05/2, df=dof )
t_cr, dof

## Значимость корреляции
Проверим значимость корреляции между переменными `sleep` и `totwrk`. Уровень значимости выберем 10%

In [ ]:
# Первый способ с использованием библиотеки scipy.stats
t_stat, p_val=scs.pearsonr(x=df['sleep'], y=df['totwrk'])
t_stat, p_val

In [ ]:
# Второй способ с использованием бибиотеки pingouin
pg.corr(x=df['sleep'], y=df['totwrk'])

10%-критическое значение $t$-распределения со степенями свободы $n-2=704$

In [ ]:
scs.t.ppf( q=1-0.10/2, df=df['sleep'].shape[0]-2 )

**Вывод**: корреляция значима

Проверим значимость корреляции между переменными `age` и `totwrk`.

In [ ]:
t_stat, p_val=scs.pearsonr(x=df['age'], y=df['totwrk'])
t_stat, p_val

In [ ]:
# Второй способ с использованием бибиотеки pengouin
pg.corr(x=df['age'], y=df['totwrk'])

**Вывод**: корреляция незначима (критическое значение тоже)

In [ ]:
# Визц=уализация корреляционной матрицы
sns.heatmap(df[['sleep', 'totwrk', 'age', 'educ']].corr(), annot = True)

## Одновыборочный тест Колмогорова-Смирнова на сравнение с нормальным распределением
Для переменной `sleep`

In [ ]:
sm.lilliefors(df['sleep'], dist='norm')

5%-критическое значение $\lambda_{cr}=0.886$.
**Вывод**: гипотеза отвергается

In [ ]:
# Эмпирическая функция распределения для sleep
sns.ecdfplot(df, x='sleep')

## Двухвыборочный тест Колмогорова-Смирнова
Разделим наблюдения по переменной `sleep` на две подвыборки относительно бинарной переменной `male`. Потестируем гипотезу об одинаковой распределённости эти выборок

In [ ]:
X = df[ df['male']==1 ]['sleep']
Y = df[ df['male']==0 ]['sleep']
stat, p_val=scs.ks_2samp(X,Y)
stat, p_val

5%-критическое значени $\lambda_{cr}=1.36$. 
**Вывод**: тестируемая гипоза не отвергается

In [ ]:
sns.ecdfplot(df, x='sleep', hue='male')

Разделим наблюдения по переменной `totwrk` на две подвыборки относительно бинарной переменной `south`. Потестируем гипотезу об одинаковой распределённости эти выборок

In [ ]:
X = df[ df['south']==1 ]['totwrk']
Y = df[ df['south']==0 ]['totwrk']
stat, p_val=scs.ks_2samp(X,Y)
stat, p_val

Критическое значение такое же. Гипотеза не отвергается


In [ ]:
# Эмпириеские функции распределения для totwrk с разделением по бинраной переменной south
sns.ecdfplot(df, x='totwrk', hue='south')